# Подготовка датасета ChEMBL

In [124]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.SaltRemover import SaltRemover
remover = SaltRemover()
from rdkit.Chem import PandasTools
#from rdkit.Chem.PandasTools import ChangeMoleculeRendering
from IPython.display import display,Image
%matplotlib inline

In [125]:
df = pd.read_csv('~/Chembl/CHEMBL25-chembl_activity-DqC439Vh39wrl-9GfJSPwktKk0J-_DmtUMN94h4Px3U=.tsv.gz', sep = '\t')
df.columns = df.columns.str.replace(' ','_')

In [126]:
#Render RDKit molecules as images
#ChangeMoleculeRendering(frame=df, renderer='PNG')

In [127]:
df['mol'] = df['Smiles'].apply(lambda x: Chem.MolFromSmiles(x))

In [128]:
df['no_salt'] = df['mol'].apply(lambda y: remover(y))

In [129]:
def _InitialiseNeutralisationReactions():
    patts= (
        # Imidazoles [S+]([#6])[#8-]
        ('[n+;H]','n'),
        # Amines
        ('[N+;!H0]','N'),
        # Carboxylic acids and alcohols
        ('[$([O-]);!$([O-][#7]);!$([O-][S+])]','O'),
        # Thiols
        ('[S-;X1]','S'),
        # Sulfonamides
        ('[$([N-;X2]S(=O)=O)]','N'),
        # Enamines
        ('[$([N-;X2][C,N]=C)]','N'),
        # Tetrazoles
        ('[n-]','[nH]'),
        # Sulfoxides
        ('[$([S-]=O)]','S'),
        # Amides
        ('[$([N-]C=O)]','N'),
        )
    return [(Chem.MolFromSmarts(x),Chem.MolFromSmiles(y,False)) for x,y in patts]

_reactions=None

def NeutraliseCharges(mol, reactions=None):
    global _reactions
    if reactions is None:
        if _reactions is None:
            _reactions=_InitialiseNeutralisationReactions()
        reactions=_reactions
    #mol = Chem.MolFromSmiles(smiles)
    replaced = False
    for i,(reactant, product) in enumerate(reactions):
        while mol.HasSubstructMatch(reactant):
            replaced = True
            rms = AllChem.ReplaceSubstructs(mol, reactant, product)
            mol = rms[0]
    if replaced:
        return (Chem.MolToSmiles(mol, 1), True)
    else:
        return (Chem.MolToSmiles(mol, 1), False)

In [130]:
df['Neutral'] = df['no_salt'].apply(lambda x: NeutraliseCharges(x))
df['neutralized'] = df['Neutral'].apply(lambda x: x[1])
df['Canonical_Smiles'] = df['Neutral'].apply(lambda x: x[0])
#print(df.iloc[1]['Smiles'])
#print(df.iloc[1]['Canonical_Smiles'])

In [131]:
df.drop(['no_salt', 'Neutral','mol' ], axis = 1, inplace = True)
print(len(df))
print(df[['Smiles','Molecule_ChEMBL_ID','Canonical_Smiles']].nunique())

4439
Smiles                3311
Molecule_ChEMBL_ID    3311
Canonical_Smiles      3309
dtype: int64


In [132]:
# Составим списки IDшников, с одинаковыми каноническими SMILES
Smile_ID_mapping = df.groupby('Canonical_Smiles', as_index = False).agg({'Molecule_ChEMBL_ID':'unique'}).rename(columns = {'Molecule_ChEMBL_ID': 'IDs_list'})

In [133]:
df1 = df.merge(Smile_ID_mapping, on = 'Canonical_Smiles')
df1['IDs'] = df1['IDs_list'].apply(lambda x: ', '.join(x))
df1.IDs.nunique()

3309

In [95]:
df1.to_csv('/Users/dueva1/Documents/Projects/PARP1/Chembl/df1_sift.csv')

# Стандартизация

## Переведем все, что можно, в нМ

In [134]:
type(df1['Standard_Units'].unique()[1])

float

In [135]:
np.isnan(df1['Standard_Units'].unique()[1])

True

In [136]:
df1[(df1['Standard_Value'] == float('nan')) & (df1['Standard_Value'] == 'pIC50')]

,Molecule_ChEMBL_ID,Molecule_Name,Molecule_Max_Phase,Molecular_Weight,#RO5_Violations,AlogP,Compound_Key,Smiles,Standard_Type,Standard_Relation,...,Document_ChEMBL_ID,Source_ID,Source_Description,Document_Journal,Document_Year,Cell_ChEMBL_ID,neutralized,Canonical_Smiles,IDs_list,IDs


In [137]:
df1['Standard_Type'].unique()

array(['IC50', 'Inhibition', 'EC50', 'Ki', 'Activity', 'FC', 'PF50', 'Kd',
       'Ratio IC50', 'Delta Tm', 'ED50', 'T1/2', 'pIC50'], dtype=object)

In [138]:
def convert_to_nM(unit, bioactivity, Standard_Type):
    k = 0
    if unit != "nM":
        if unit == "uM":
            val = float(bioactivity)*1000
        elif unit == "10'8nM":
            val = float(bioactivity)*100000000
        elif unit == "10'13nM":
            val = float(bioactivity)*10000000000000
#        elif (np.isnan(unit)) & (Standard_Type == "pIC50"):
#            print(k)
#            val = 10^(float(bioactivity))
        else:
            val = float('NaN')
            #print ('unit not recognized...', unit)  
        return val
    else:
        return bioactivity

In [139]:
bioactivity_nM = []

for i, row in df1.iterrows():
    bioact_nM = convert_to_nM(row['Standard_Units'], row['Standard_Value'], row['Standard_Type'])
    bioactivity_nM.append(bioact_nM)
    
df1['value'] = bioactivity_nM
df1['units'] = 'nM'

df2 = df1.dropna(axis = 0, subset = ['value'])
df2['pvalue'] = np.log10(df2['value'])

print(len(df2)-len(df1))

-440


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [140]:
df1.groupby(['BAO_Label','Standard_Type','Standard_Relation'], as_index = True)['Standard_Value'].describe(percentiles = [.5])

count           mean  \
BAO_Label             Standard_Type Standard_Relation                          
assay format          IC50          '<'                  21.0      34.285714   
                                    '='                 246.0     970.868191   
                                    '>'                  22.0    8745.454545   
                      Inhibition    '='                   1.0      90.000000   
                      Ratio IC50    '='                   1.0      10.000000   
cell-based format     IC50          '<'                   1.0      30.000000   
                                    '='                 174.0    1182.821552   
                                    '>'                  15.0   20733.333333   
                      Activity      '='                   8.0      11.982500   
                      EC50          '='                 281.0    8071.726655   
                                    '>'                   8.0    1000.000000   
                      ED50          '='                  34.0       1.469118   
                                    '>'                   9.0      30.000000   
                      FC            '='                   6.0       1.950000   
                      PF50          '='                  69.0     386.334203   
single protein format IC50          '<'                   2.0   12000.000000   
                                    '='                1640.0    7654.576700   
                                    '>'                 170.0   32101.427882   
                      Inhibition    '<'                  20.0      24.000000   
                                    '='                  95.0      51.906211   
                                    '>'                  20.0      60.000000   
                      Ratio IC50    '='                   5.0       5.000000   
                      Activity      '='                   4.0      22.327500   
                                    '>'                   4.0      50.000000   
                      EC50          '='                  47.0     374.595745   
                                    '>'                   2.0    2000.000000   
                      Delta Tm      '='                  56.0       1.797143   
                      Kd            '<'                  38.0      22.105263   
                                    '='                 107.0    5457.095701   
                                    '>'                   2.0  500000.000000   
                      Ki            '<'                   4.0       5.000000   
                                    '='                1165.0     250.328764   
                                    '>'                   3.0    9833.333333   
                      T1/2          '='                   5.0       0.192667   

                                                                std  \
BAO_Label             Standard_Type Standard_Relation                 
assay format          IC50          '<'                   15.352989   
                                    '='                 1995.801422   
                                    '>'                21787.408493   
                      Inhibition    '='                         NaN   
                      Ratio IC50    '='                         NaN   
cell-based format     IC50          '<'                         NaN   
                                    '='                 2589.271665   
                                    '>'                32351.234612   
                      Activity      '='                   12.016158   
                      EC50          '='                28265.661007   
                                    '>'                    0.000000   
                      ED50          '='                    1.690979   
                                    '>'                    0.000000   
                      FC            '='                    0.383406   
                      PF50          '='

# Разберемся с типами эссеев

In [141]:
df1['Standard_Type'].unique()

array(['IC50', 'Inhibition', 'EC50', 'Ki', 'Activity', 'FC', 'PF50', 'Kd',
       'Ratio IC50', 'Delta Tm', 'ED50', 'T1/2', 'pIC50'], dtype=object)

## Узнаем, что за EC50 в Single Protein Format

In [142]:
Assay_Chembl_ID_to_check = set(df2[(df2['BAO_Label'] == 'single protein format')\
                                   & (df2['Standard_Type'] == 'EC50')]['Assay_ChEMBL_ID'].tolist())
Assay_Chembl_ID_to_check

{'CHEMBL3110586', 'CHEMBL761607', 'CHEMBL762474', 'CHEMBL829884'}

In [143]:
df2[df2['Assay_ChEMBL_ID'].isin(Assay_Chembl_ID_to_check)].\
groupby('Assay_ChEMBL_ID', as_index = False).\
agg({'Assay_Description':'unique'}).values

array([['CHEMBL3110586',
        array(['Inhibition of ARTD1 (unknown origin)'], dtype=object)],
       ['CHEMBL761607',
        array(['Concentration required to inhibit human recombinant Poly (ADP-ribose) polymerase 1 was determined using cell protection assay'],
      dtype=object)],
       ['CHEMBL762474',
        array(['Concentration required to inhibit human recombinant PARP-1 was determined using cell protection assay'],
      dtype=object)],
       ['CHEMBL829884',
        array(['Effective concentration against poly ADP-ribose polymerase-1 determined using the cell protection assay'],
      dtype=object)]], dtype=object)

In [144]:
У CHEMBL3110586 - странный  Assay Descroption 'Inhibition of ARTD1 (unknown origin)' - дропнем
Для остальных Assay_ChEMBL_ID (CHEMBL761607, CHEMBL762474, CHEMBL829884) изменим BAO_Label на 'cell-based format'

SyntaxError: invalid syntax (<ipython-input-144-8e3f103cbc93>, line 1)

In [145]:
df2['New_BAO_Label'] = np.where(df2['Assay_ChEMBL_ID'].isin(Assay_Chembl_ID_to_check),
                                'cell-based format', df2['BAO_Label'])

df3 = df2[~df2.Assay_ChEMBL_ID.isin(['CHEMBL3110586'])]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Проверим, что такое 'Assay format' в колонке ВAO_Lable

In [146]:
k = df3[(df3['BAO_Label'] == 'assay format') & (df3['Standard_Type'] == 'IC50')]
aids = set(k['Assay_ChEMBL_ID'].to_list())


k.groupby('Assay_ChEMBL_ID', as_index = False).\
agg({'Assay_Description':'unique'}).values

array([['CHEMBL3107467',
        array(['Inhibition of recombinant human GST-fused PARP-1 expressed in Escherichia coli after 30 mins by fluorescence assay'],
      dtype=object)],
       ['CHEMBL3107875',
        array(['Inhibition of GST-tagged recombinant human PARP-1 expressed in Escherichia coli after 30 mins by fluorescence-based assay'],
      dtype=object)],
       ['CHEMBL3420116',
        array(['Inhibition of hexahistidine-tagged full length human recombinant ARTD1 expressed in Escherichia coli BL21(DE3) assessed as Inhibition of ADP-ribosyltransferase activity incubated for 15 mins using biotin-NAD+ by chemiluminescence detection based assay'],
      dtype=object)],
       ['CHEMBL3579452',
        array(['Inhibition of full length PARP1 (unknown origin) expressed in Escherichia coli BL21 (DE3) assessed as reduction in ADP-ribosyl transferase activity using NAD+ by chemiluminescence detection based assay'],
      dtype=object)],
       ['CHEMBL3887955',
        array(['ELIS

In [147]:
df3['New_BAO_Label'] = np.where(df3['Assay_ChEMBL_ID'].isin(aids),
                                'single protein format', df3['New_BAO_Label'])
df3[df3['Assay_ChEMBL_ID'].isin(aids)][['New_BAO_Label', 'BAO_Label']].head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,New_BAO_Label,BAO_Label
234,single protein format,assay format
238,single protein format,assay format
239,single protein format,assay format
240,single protein format,assay format
380,single protein format,assay format


## Проверим, что такое 'Standard_Type' == 'Activity' в 'cell-based format'

In [148]:
k1 = df3[(df3['BAO_Label'] == 'cell-based format') & (df3['Standard_Type'] == 'Activity')]

ass1 = set(k1['Assay_ChEMBL_ID'].to_list())

k1.groupby('Assay_ChEMBL_ID', as_index = False).\
agg({'Assay_Description':'unique'}).values

#Здесь мы ничего с этим не сделали, но в дальнейшем значение 'Activity' поменяем на EC50
#(хотя, возможно, стоит удалить, ибо описание странное)

array([['CHEMBL952238',
        array(['Inhibition of PARP1 in human HCT116 cells assessed as assessed as chemosensitization measured by drug concentration that reduced half the amount of streptozotocin required to inhibit 90% cell growth after 24 hrs'],
      dtype=object)]], dtype=object)

## Посмотрим, что такое ED50

In [149]:
k = df3[(df3['BAO_Label'] == 'cell-based format') & (df3['Standard_Type'] == 'ED50')]

ass1 = set(k['Assay_ChEMBL_ID'].to_list())

k.groupby('Assay_ChEMBL_ID', as_index = False).\
agg({'Assay_Description':'unique'}).values

array([['CHEMBL1111553',
        array(['Inhibition of PARP1 expressed in CHO-K1 cells'], dtype=object)],
       ['CHEMBL3061927',
        array(['Inhibition of PARP-1 (unknown origin) expressed in CHO-K1 cells assessed as depletion of NAD(P)H after 2 hr by CCK-8 assay'],
      dtype=object)]], dtype=object)

In [150]:
df3 = df3[~df3.Assay_ChEMBL_ID.isin(['CHEMBL1111553'])] #В первой версии это удалали, но кажется, можно оставить, тк описание легитимное
#https://sci-hub.se/10.1016/j.bmcl.2010.02.014

## Что за IC50 в клеточных эссеях?

In [151]:
k = df3[(df3['BAO_Label'] == 'cell-based format') & (df3['Standard_Type'] == 'IC50')]

k.groupby('Assay_ChEMBL_ID', as_index = False).\
agg({'Assay_Description':'unique'}).values

array([['CHEMBL1060805',
        array(['Inhibition of PARP1 in human HeLa cells by microplate scintillation counting'],
      dtype=object)],
       ['CHEMBL3607349',
        array(['Inhibition of PARP1 in human HeLa cells assessed as reduction of H2O2-induced PAR formation preincubated for 30 mins followed by H2O2 addition measured after 15 mins by immunocytochemical analysis'],
      dtype=object)],
       ['CHEMBL3738636',
        array(['Inhibition of PARP1 in human G7 cells incubated for 60 mins by immunofluorescence assay'],
      dtype=object)],
       ['CHEMBL3738637',
        array(['Inhibition of PARP1 in human T98G cells incubated for 60 mins by immunofluorescence assay'],
      dtype=object)],
       ['CHEMBL3743825',
        array(['Inhibition of full length human PARP1 expressed in Baculovirus infected Sf9 insect cells using activated DNA as substrate after 1 hr by streptavidin-horseradish peroxidase-based luminescence assay'],
      dtype=object)],
       ['CHEMBL380723

In [152]:
ass2 = ['CHEMBL4003517',
        'CHEMBL4055669',# не уверена, что это правильное что-то https://sci-hub.se/10.1016/j.bmc.2017.05.058, можно удалить
        'CHEMBL3807232',
        'CHEMBL859247',
        'CHEMBL3743825',
        'CHEMBL901413', # https://sci-hub.se/10.1016/j.bmcl.2005.10.099 https://sci-hub.se/10.1016/j.bmcl.2006.10.010, можно удалить
        'CHEMBL1060805']

In [153]:
#Перенесем в Single protein format
df3['New_BAO_Label'] = np.where(df3['Assay_ChEMBL_ID'].isin(ass2),
                                'single protein format', df3['New_BAO_Label'])

In [154]:
df3[df3.Assay_ChEMBL_ID.isin(ass2)].head()

,Molecule_ChEMBL_ID,Molecule_Name,Molecule_Max_Phase,Molecular_Weight,#RO5_Violations,AlogP,Compound_Key,Smiles,Standard_Type,Standard_Relation,...,Document_Year,Cell_ChEMBL_ID,neutralized,Canonical_Smiles,IDs_list,IDs,value,units,pvalue,New_BAO_Label
1,CHEMBL557175,NaN,0,469.36,0.0,3.28,7,Br.NCc1nc(cs1)c2ccc3[nH]c4c5CCCc5c6C(=O)NC(=O)...,IC50,'=',...,2007,CHEMBL3307756,False,NCc1nc(-c2ccc3[nH]c4c5c(c6c(c4c3c2)C(=O)NC6=O)...,[CHEMBL557175],CHEMBL557175,42.0,nM,1.623249,single protein format
79,CHEMBL571900,NaN,0,319.41,0.0,2.79,4,O=C1NCCc2c1ccc3[nH]cc(CCNCc4ccccc4)c23,IC50,'=',...,2009,CHEMBL3308376,False,O=C1NCCc2c1ccc1[nH]cc(CCNCc3ccccc3)c21,[CHEMBL571900],CHEMBL571900,110.0,nM,2.041393,single protein format
80,CHEMBL559994,NaN,0,370.46,0.0,3.33,8,O=C1NCCc2c1ccc3[nH]cc(CCNCc4cccc5cccnc45)c23,IC50,'>',...,2009,CHEMBL3308376,False,O=C1NCCc2c1ccc1[nH]cc(CCNCc3cccc4cccnc34)c21,[CHEMBL559994],CHEMBL559994,500.0,nM,2.698970,single protein format
81,CHEMBL564183,NaN,0,285.39,0.0,2.34,9,CCN(CC)CCc1c[nH]c2ccc3C(=O)NCCc3c12,IC50,'=',...,2009,CHEMBL3308376,False,CCN(CC)CCc1c[nH]c2ccc3c(c12)CCNC3=O,[CHEMBL564183],CHEMBL564183,130.0,nM,2.113943,single protein format
107,CHEMBL247374,NaN,0,305.34,0.0,2.15,2,NCc1ccc2[nH]c3c4CCCc4c5C(=O)NC(=O)c5c3c2c1,IC50,'=',...,2007,CHEMBL3307756,False,NCc1ccc2[nH]c3c4c(c5c(c3c2c1)C(=O)NC5=O)CCC4,[CHEMBL247374],CHEMBL247374,18.0,nM,1.255273,single protein format


In [155]:
df3.groupby(['New_BAO_Label','Standard_Type', 'Standard_Relation'], 
            as_index = True)['value'].describe(percentiles = [.5])

count          mean  \
New_BAO_Label         Standard_Type Standard_Relation                         
cell-based format     Activity      '='                   8.0  1.198250e+04   
                      EC50          '='                 327.0  6.990034e+03   
                                    '>'                  10.0  1.200000e+03   
                      ED50          '='                  18.0  1.584444e+03   
                                    '>'                   8.0  3.000000e+04   
                      IC50          '='                  85.0  1.094605e+03   
                                    '>'                   7.0  1.000000e+04   
single protein format IC50          '<'                  24.0  1.031250e+03   
                                    '='                1974.0  3.078030e+10   
                                    '>'                 200.0  2.945321e+04   
                      Kd            '<'                  38.0  2.210526e+01   
                                    '='                 107.0  5.457096e+03   
                                    '>'                   2.0  5.000000e+05   
                      Ki            '<'                   4.0  5.000000e+00   
                                    '='                1165.0  2.503288e+02   
                                    '>'                   3.0  9.833333e+03   

                                                                std  \
New_BAO_Label         Standard_Type Standard_Relation                 
cell-based format     Activity      '='                1.201616e+04   
                      EC50          '='                2.633305e+04   
                                    '>'                6.324555e+02   
                      ED50          '='                1.104053e+03   
                                    '>'                0.000000e+00   
                      IC50          '='                2.444632e+03   
                                    '>'                0.000000e+00   
single protein format IC50          '<'                3.391209e+03   
                                    '='                1.367553e+12   
                                    '>'                7.577040e+04   
                      Kd            '<'                9.907108e+00   
                                    '='                2.337158e+04   
                                    '>'                0.000000e+00   
                      Ki            '<'                0.000000e+00   
                                    '='                2.967774e+03   
                                    '>'                2.886751e+02   

                                                             min       50%  \
New_BAO_Label         Standard_Type Standard_Relation                        
cell-based format     Activity      '='                   380.00    9250.0   
                      EC50          '='                     0.90      70.0   
                                    '>'                  1000.00    1000.0   
                      ED50          '='                   320.00    1275.0   
                                    '>'                 30000.00   30000.0   
                      IC50          '='                     0.20     130.0   
                                    '>'                 10000.00   10000.0   
single protein format IC50          '<'                    20.00      40.0   
                                    '='                     0.31      80.5   
                                    '>'                   100.00   19000.0   
                      Kd            '<'                    10.00      30.0   
                                    '='                     0.24      54.0   
                                    '>'                500000.00  500000.0   
                      Ki            '<'                     5.00       5.0   
                                    '='                     0.10      16.0   
                            

In [156]:
#Поменяем у оставшихся в 'cell-based format'значения 'Activity','ED50', 'IC50' на EC50

df3['New_Standard_Type'] = np.where(((df3['Standard_Type'].isin(['ED50', 'IC50', 'Activity'])
                                     & df3['New_BAO_Label'] == 'cell-based format')),
                                'EC50', df3['Standard_Type'])

for k in ['ED50', 'IC50', 'Activity']:
    df3['New_Standard_Type'] = np.where(((df3['Standard_Type'] == k) & (df3['New_BAO_Label'] == 'cell-based format')),'EC50', df3['New_Standard_Type'])

In [157]:
df3.groupby(['New_BAO_Label','New_Standard_Type', 'Standard_Relation'], as_index = True)['value'].describe(percentiles = [.5])

count  \
New_BAO_Label         New_Standard_Type Standard_Relation           
cell-based format     EC50              '='                 438.0   
                                        '>'                  25.0   
single protein format IC50              '<'                  24.0   
                                        '='                1974.0   
                                        '>'                 200.0   
                      Kd                '<'                  38.0   
                                        '='                 107.0   
                                        '>'                   2.0   
                      Ki                '<'                   4.0   
                                        '='                1165.0   
                                        '>'                   3.0   

                                                                   mean  \
New_BAO_Label         New_Standard_Type Standard_Relation                 
cell-based format     EC50              '='                5.714983e+03   
                                        '>'                1.288000e+04   
single protein format IC50              '<'                1.031250e+03   
                                        '='                3.078030e+10   
                                        '>'                2.945321e+04   
                      Kd                '<'                2.210526e+01   
                                        '='                5.457096e+03   
                                        '>'                5.000000e+05   
                      Ki                '<'                5.000000e+00   
                                        '='                2.503288e+02   
                                        '>'                9.833333e+03   

                                                                    std  \
New_BAO_Label         New_Standard_Type Standard_Relation                 
cell-based format     EC50              '='                2.296944e+04   
                                        '>'                1.253435e+04   
single protein format IC50              '<'                3.391209e+03   
                                        '='                1.367553e+12   
                                        '>'                7.577040e+04   
                      Kd                '<'                9.907108e+00   
                                        '='                2.337158e+04   
                                        '>'                0.000000e+00   
                      Ki                '<'                0.000000e+00   
                                        '='                2.967774e+03   
                                        '>'                2.886751e+02   

                                                                 min  \
New_BAO_Label         New_Standard_Type Standard_Relation              
cell-based format     EC50              '='                     0.20   
                                        '>'                  1000.00   
single protein format IC50              '<'                    20.00   
                                        '='                     0.31   
                                        '>'                   100.00   
                      Kd                '<'                    10.00   
                                        '='                     0.24   
                                        '>'                500000.00   
                      Ki                '<'                     5.00   
                                        '='                     0.10   
                                        '>'                  9500.00   

                                                                50%  \
New_BAO_Label         New_Standard_Type Standard_Relation             
cell-based format     EC50              '='                   100.0   
                                        '>'                 

In [158]:
print(len(df3))
print(len(df1))
print(len(df2))

3980
4439
3999


In [159]:
k = df3[(df3['BAO_Label'] == 'cell-based format') & (df3['Standard_Type'] == 'EC50')]

k.groupby('Assay_ChEMBL_ID', as_index = False).\
agg({'Assay_Description':'unique'}).values

array([['CHEMBL1010817',
        array(['Inhibition of PARP1 in human C41 cells'], dtype=object)],
       ['CHEMBL1042277',
        array(['Inhibition of PARP1 in human C41 cells after 30 mins by cell-based assay'],
      dtype=object)],
       ['CHEMBL1060926',
        array(['Inhibition of PARP1 in human C41 cells by DAPI staining-based FITC analysis'],
      dtype=object)],
       ['CHEMBL1073570',
        array(['Inhibition of PARP1 in human C41 cells'], dtype=object)],
       ['CHEMBL1074648',
        array(['Inhibition of PARP1 in human HeLa cells assessed as inhibition of hydrogen peroxide-induced poly(ADP-ribosyl)ation'],
      dtype=object)],
       ['CHEMBL1106195',
        array(['Inhibition of PARP1 in human C41 cells by FITC-conjugated DAPI staining'],
      dtype=object)],
       ['CHEMBL2061719',
        array(['Inhibition of PARP1 in H202-stimulated human C41 cells incubated for 30 mins prior to H2O2-treatment measured after 10 mins by FITC-based immunostaining'],
     

# Разберемся с активными и неактивными

## Ki Kd

In [160]:
th=50

Будем предполагать, что если Ki или Kd меньше трешхолда, то и IC50 будет меньше трешхолда и эти данные можно использовать

In [162]:
kikd = df3[((df3['New_Standard_Type']  == 'Ki') | (df3['New_Standard_Type']  == 'Kd'))]

print('len unique', kikd['IDs'].nunique())
print('len', len(kikd))

len unique 1200
len 1319


In [163]:
# Проверим, есть ли такие измерения, они неинформативны
print(len(kikd[(kikd['Standard_Relation']  == "'>'") & (kikd['value'] < th)]))
print(len(kikd[(kikd['Standard_Relation']  == "'<'") & (kikd['value'] > th)].index))

0
0


In [164]:
#Сколько всего строк со знаками > или <
len(kikd[~(kikd['Standard_Relation'] == "'='")])

47

In [165]:
#Функция возвращает список айдишников молекул, для которых одни измерения показали, что она активна, а другие - что неактивна
#А также датафрейм со статистикой для таких молекул

def find_dublicates(df):

    def mk_compare(t):
        def compare(a):
            
            #Возвращаем 0 если по всем измерениям молекула активна или неактивна
            if a.apply(lambda x: x >= t).all() or a.apply(lambda x: x <= t).all():
                return 0
            else:
                return 1
        return compare

    fg = pd.DataFrame(df.groupby('IDs')['value'].apply(mk_compare(th)))
    fg.reset_index(inplace = True)
    j = fg[fg['value'] == 1]['IDs'].to_list()

    j = set(j)


    des = df[df['IDs'].isin(j)][['value','IDs']].groupby('IDs', as_index = True).describe()
    des.reset_index(inplace = True)
    
    return j, des

In [166]:
j1, des1 = find_dublicates(kikd)
j1

{'CHEMBL3605992'}

In [167]:
des1

IDs value                                                 
                 count  mean        std   min    25%   50%    75%   max
0  CHEMBL3605992   2.0  52.5  31.819805  30.0  41.25  52.5  63.75  75.0

In [168]:
kikd[kikd.IDs == 'CHEMBL3605992'].index
kikd = kikd.drop(kikd[kikd.IDs == 'CHEMBL3605992'].index)
len(kikd)

1317

In [170]:
#Отберем только неактивные

m = kikd[(kikd['value'] >= th) &
     ((kikd['Standard_Relation']  == "'='") | (kikd['Standard_Relation']  == "'>'"))].index
len(m)

final_kikd = kikd.loc[m]
len(final_kikd)

471

In [171]:
final_kikd['binary_activity'] = 0

In [172]:
# Дропнем дубликаты

final_kikd = final_kikd[['IDs', 'New_BAO_Label', 'New_Standard_Type', 'binary_activity','Canonical_Smiles']].drop_duplicates()

In [173]:
len(final_kikd)

456

In [174]:
final_kikd[['IDs', 'New_BAO_Label', 'New_Standard_Type', 'binary_activity','Canonical_Smiles']].nunique()

IDs                  456
New_BAO_Label          1
New_Standard_Type      2
binary_activity        1
Canonical_Smiles     456
dtype: int64

# IC50

In [176]:
ic50 = df3[(df3.New_BAO_Label == 'single protein format') &  (df3.New_Standard_Type == 'IC50')]
print('total ic50 =', len(ic50))

th = 50
print(len(ic50[(ic50['Standard_Relation']  == "'>'") & (ic50['value'] <= th)]))

m = ic50[(ic50['Standard_Relation']  == "'<'") & (ic50['value'] > th)].index
ic50.drop(m, inplace = True)

print(len(ic50))

total ic50 = 2198
0
2196


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [177]:
j1, des1 = find_dublicates(ic50)
des1

IDs value                                     \
                               count         mean          std      min   
0                CHEMBL1086580  15.0   916.946000   865.215138  11.0000   
1                CHEMBL1094636  10.0    40.730000    49.444841   2.0000   
2                CHEMBL1173055   9.0   132.417378   186.141554   0.8044   
3   CHEMBL190434, CHEMBL426270   4.0   210.500000   393.429790   1.0000   
4                 CHEMBL194482   2.0    82.500000    51.618795  46.0000   
5                 CHEMBL249813   3.0    57.000000    19.052559  46.0000   
6                 CHEMBL251027   3.0   109.633333   168.363902   8.9000   
7                 CHEMBL338790   2.0   103.000000   108.894444  26.0000   
8                 CHEMBL339695   2.0  3504.500000  4943.383507   9.0000   
9                 CHEMBL372303  12.0   144.412500   163.914411  10.9000   
10               CHEMBL3764816   3.0    48.740667    82.497003   1.1000   
11               CHEMBL3912508   2.0    59.370000    55.677588  20.0000   
12               CHEMBL3960883   2.0    51.985000    45.233621  20.0000   
13                CHEMBL506871  15.0    65.947400   121.528424   3.3000   
14                CHEMBL521686  27.0    17.773704    25.390845   0.9000   

                                             
          25%       50%        75%      max  
0    120.0000  1180.000  1370.0000  2300.00  
1     10.0925    23.350    34.3250   132.00  
2      3.1620    39.800   301.0000   500.00  
3      1.0000    20.500   230.0000   800.00  
4     64.2500    82.500   100.7500   119.00  
5     46.0000    46.000    62.5000    79.00  
6     12.4500    16.000   160.0000   304.00  
7     64.5000   103.000   141.5000   180.00  
8   1756.7500  3504.500  5252.2500  7000.00  
9     50.1950   110.000   125.0500   481.00  
10     1.1110     1.122    72.5610   144.00  
11    39.6850    59.370    79.0550    98.74  
12    35.9925    51.985    67.9775    83.97  
13     5.0900    10.000    35.3400   354.81  
14     3.7950     7.000    13.7500    86.32

выбросим молекулы, которые измерялись меньше 4 раз, остальные проверим глазами

In [178]:
to_drop = [
 'CHEMBL194482',
 'CHEMBL249813',
 'CHEMBL251027',
 'CHEMBL338790',
 'CHEMBL339695',
 'CHEMBL3764816',
 'CHEMBL3912508',
 'CHEMBL3960883']

ic50  = ic50[~ic50['IDs'].isin(to_drop)]

In [179]:
# Проверим
check = []

for k in j1:
    if k not in to_drop:
        check.append(k)

        #107/2683:
ic50[ic50['IDs'].isin(check)].groupby('IDs')['value'].describe().reset_index().sort_values('count', ascending = False)

,IDs,count,mean,std,min,25%,50%,75%,max
6,CHEMBL521686,27.0,17.773704,25.390845,0.9000,3.7950,7.00,13.750,86.32
0,CHEMBL1086580,15.0,916.946000,865.215138,11.0000,120.0000,1180.00,1370.000,2300.00
5,CHEMBL506871,15.0,65.947400,121.528424,3.3000,5.0900,10.00,35.340,354.81
4,CHEMBL372303,12.0,144.412500,163.914411,10.9000,50.1950,110.00,125.050,481.00
1,CHEMBL1094636,10.0,40.730000,49.444841,2.0000,10.0925,23.35,34.325,132.00
2,CHEMBL1173055,9.0,132.417378,186.141554,0.8044,3.1620,39.80,301.000,500.00
3,"CHEMBL190434, CHEMBL426270",4.0,210.500000,393.429790,1.0000,1.0000,20.50,230.000,800.00


In [180]:
#Собственно, проверяем глазами
ic50[ic50['IDs'] == 'CHEMBL521686'][['Assay_ChEMBL_ID', 'value', 'Standard_Relation']].sort_values(by = 'value')

,Assay_ChEMBL_ID,value,Standard_Relation
2275,CHEMBL3887957,0.90,'='
2266,CHEMBL3743825,1.00,'='
2254,CHEMBL3995357,1.38,'='
2270,CHEMBL3995357,1.40,'='
2252,CHEMBL3767923,1.94,'='
2279,CHEMBL4029615,2.09,'='
2288,CHEMBL4055669,3.59,'='
2261,CHEMBL2330729,4.00,'='
2258,CHEMBL3107467,4.50,'='
2284,CHEMBL982809,5.00,'='


In [104]:
#ic50[ic50['IDs'] == 'CHEMBL1086580'][['Assay_ChEMBL_ID', 'value', 'Standard_Relation']].sort_values(by = 'value')

In [105]:
#107/2686:
#ic50[ic50['IDs'] == 'CHEMBL506871'][['Assay_ChEMBL_ID', 'value', 'Standard_Relation']].sort_values(by = 'value')
#107/2687: ic50[ic50['IDs'] == 'CHEMBL372303'][['Assay_ChEMBL_ID', 'value', 'Standard_Relation']].sort_values(by = 'value')
#107/2688: ic50[ic50['IDs'] == 'CHEMBL1094636'][['Assay_ChEMBL_ID', 'value', 'Standard_Relation']].sort_values(by = 'value')
#107/2689: ic50[ic50['IDs'] == 'CHEMBL1173055'][['Assay_ChEMBL_ID', 'value', 'Standard_Relation']].sort_values(by = 'value')
#107/2690: ic50[ic50['IDs'] == 'CHEMBL190434, CHEMBL426270'][['Assay_ChEMBL_ID', 'value', 'Standard_Relation']].sort_values(by = 'value')

In [106]:
# Если у молекулы было много измерений, но в каком-то эссее результаты расходятся с большинством, то мы полностью удаляем эссей

to_drop = ['CHEMBL3995358','CHEMBL3371141','CHEMBL3887956',
        'CHEMBL2317526',
        'CHEMBL4029618', 'CHEMBL3995358',
        'CHEMBL3995357','CHEMBL1176193',
        'CHEMBL3995359',
        'CHEMBL3995358','CHEMBL3428885',
        'CHEMBL828154']

ic50_1  = ic50[~ic50['Assay_ChEMBL_ID'].isin(to_drop)]

In [107]:
#Для каждой молекулы получим подсчет количества ее измерений
ic50_desc = ic50_1.groupby('IDs')['value'].describe().reset_index().sort_values('count', ascending = False)
#ic50_desc

#Добавим эти данные в наш датафрейм
ic50_1 = ic50_1.merge(ic50_desc[['IDs', 'count']], on = 'IDs', how = 'inner', validate = 'many_to_one')

In [108]:
def assign_binary_activity(df):

    def mk_compare(t):
        def compare(a):
            if a.apply(lambda x: x >= t).all():
                return 0
            elif a.apply(lambda x: x <= t).all():
                return 1
            else:
                return float('NaN')
        return compare

    fg = pd.DataFrame(df.groupby('IDs')['value'].apply(mk_compare(th)))
    fg.reset_index(inplace = True)
    fg.rename(columns = {'value': 'binary_activity'}, inplace = True)
    
    return fg

In [109]:
result = assign_binary_activity(ic50_1)

In [110]:
result.head()

,IDs,binary_activity
0,CHEMBL105765,0
1,CHEMBL105925,0
2,CHEMBL105995,1
3,CHEMBL106154,0
4,CHEMBL106298,0


In [111]:
len(result[result.binary_activity == 1])

733

In [112]:
#ic50_1.drop('binary_activity', axis = 1, inplace = True)
ic50_1 = ic50_1.merge(result, on = 'IDs', how = 'inner', validate = 'many_to_one')

print(len(ic50_1))

ic50_1.groupby('IDs')['binary_activity'].describe().reset_index().sort_values('count', ascending = False).head()

2047


,IDs,count,mean,std,min,25%,50%,75%,max
1539,CHEMBL521686,18.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
23,CHEMBL1086580,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1831,CHEMBL81977,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1527,CHEMBL506871,8.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
1443,CHEMBL45245,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
check = ic50_1.groupby('IDs')['binary_activity'].describe().reset_index().sort_values('count', ascending = False)

In [114]:
check[check['min'] < check['max']]

,IDs,count,mean,std,min,25%,50%,75%,max


In [115]:
check[check['min'] > check['max']]

,IDs,count,mean,std,min,25%,50%,75%,max


In [116]:
final_ic50 = ic50_1[['IDs', 'New_BAO_Label', 'New_Standard_Type', 'binary_activity','Canonical_Smiles']].drop_duplicates()


In [117]:
len(final_ic50)

1847

In [118]:
final_ic50_1 = ic50_1[['IDs']].drop_duplicates()
len(final_ic50_1)

1847

# Cell-based

Также как и в kikd будем предполагать, что если соединение активно в клеточном эссее, то и IC50 будет <= 50 нМ

In [119]:
cell = df3[df3['New_BAO_Label'] == 'cell-based format']

print(cell['IDs'].nunique())

print(len(cell[(cell['Standard_Relation']  == "'>'") & (cell['value'] < th)]))
print(len(cell[(cell['Standard_Relation']  == "'<'") & (cell['value'] > th)]))

len(cell[~(cell['Standard_Relation'] == "'='")])

412
0
0


25

In [120]:
l, des2 = find_dublicates(cell)

des2

IDs value                                               \
                  count          mean           std   min        25%   
0   CHEMBL2407969   2.0  50010.000000  70696.535983  20.0  25015.000   
1   CHEMBL2407970   2.0  50010.000000  70696.535983  20.0  25015.000   
2   CHEMBL2407982   2.0  21505.000000  30398.520523  10.0  10757.500   
3   CHEMBL2407987   2.0  22005.000000  31105.627304  10.0  11007.500   
4   CHEMBL2407988   2.0  38510.000000  54433.080016  20.0  19265.000   
5   CHEMBL2407989   2.0  13010.000000  18370.634175  20.0   6515.000   
6    CHEMBL455590   2.0    951.100000   1341.947249   2.2    476.650   
7    CHEMBL497571   2.0    193.450000    263.821540   6.9    100.175   
8    CHEMBL506871   4.0  40751.000000  79511.894297   2.0      2.000   
9    CHEMBL508796   2.0   9664.900000  13626.089095  29.8   4847.350   
10   CHEMBL521686   6.0   2678.616667   6526.158501   1.6      1.825   
11   CHEMBL525167   2.0    193.650000    263.538697   7.3    100.475   

                                   
         50%        75%       max  
0   50010.00  75005.000  100000.0  
1   50010.00  75005.000  100000.0  
2   21505.00  32252.500   43000.0  
3   22005.00  33002.500   44000.0  
4   38510.00  57755.000   77000.0  
5   13010.00  19505.000   26000.0  
6     951.10   1425.550    1900.0  
7     193.45    286.725     380.0  
8    1501.00  42250.000  160000.0  
9    9664.90  14482.450   19300.0  
10      4.25     46.500   16000.0  
11    193.65    286.825     380.0

In [121]:
set(des2.IDs.to_list())

{'CHEMBL2407969',
 'CHEMBL2407970',
 'CHEMBL2407982',
 'CHEMBL2407987',
 'CHEMBL2407988',
 'CHEMBL2407989',
 'CHEMBL455590',
 'CHEMBL497571',
 'CHEMBL506871',
 'CHEMBL508796',
 'CHEMBL521686',
 'CHEMBL525167'}

In [122]:
# Удалим результаты с с измерениями меньше 3 раз

to_drop = ['CHEMBL2407969',
 'CHEMBL2407970',
 'CHEMBL2407982',
 'CHEMBL2407987',
 'CHEMBL2407988',
 'CHEMBL2407989',
 'CHEMBL455590',
 'CHEMBL497571',
 'CHEMBL508796',
 'CHEMBL525167']

cell = cell[~cell['IDs'].isin(to_drop)]

In [123]:
cell[cell.IDs == 'CHEMBL521686'][['value', 'Assay_ChEMBL_ID', 'Standard_Relation']]

,value,Assay_ChEMBL_ID,Standard_Relation
2263,6.0,CHEMBL985473,'='
2264,1.6,CHEMBL3738637,'='
2265,1.6,CHEMBL3738636,'='
2277,60.0,CHEMBL2412860,'='
2278,16000.0,CHEMBL2412861,'='
2291,2.5,CHEMBL3107009,'='


In [124]:
cell[cell.IDs == 'CHEMBL506871'][['value', 'Assay_ChEMBL_ID', 'Standard_Relation']]

,value,Assay_ChEMBL_ID,Standard_Relation
539,2.0,CHEMBL1042277,'='
548,160000.0,CHEMBL2412861,'='
557,2.0,CHEMBL1010817,'='
558,3000.0,CHEMBL2412860,'='


In [125]:
#Удалим результаты эссеев, в которых значения для контрольных молекул отличались от большинства
aid = set(['CHEMBL2412860', 'CHEMBL2412861',
       'CHEMBL1042277', 'CHEMBL2412861', 'CHEMBL1010817', 'CHEMBL2412860'])

cell = cell[~cell['Assay_ChEMBL_ID'].isin(aid)]

In [126]:
m = cell[(cell['value'] <= th) &
     ((cell['Standard_Relation']  == "'='") | (cell['Standard_Relation']  == "'<'"))].index

cell = cell.loc[m]

cell['binary_activity'] = 1

In [127]:
print(cell['IDs'].nunique())

122


In [128]:
check = cell.groupby('IDs')['binary_activity'].describe().reset_index().sort_values('count', ascending = False)
#check

In [131]:
#107/2868:
final_cell = cell[['IDs', 'New_BAO_Label', 'New_Standard_Type', 'binary_activity','Canonical_Smiles']].drop_duplicates()

In [132]:
len(final_cell)

122

# Final

In [133]:
#Проверим, есть ли записи с одинаковыми айдишниками
dup= final_ic50.merge(final_kikd, on = 'IDs', how = 'inner', validate = 'many_to_one')
dup

,IDs,New_BAO_Label_x,New_Standard_Type_x,binary_activity_x,Canonical_Smiles_x,New_BAO_Label_y,New_Standard_Type_y,binary_activity_y,Canonical_Smiles_y
0,CHEMBL1086580,single protein format,IC50,0,Oc1nc(-c2ccc(C(F)(F)F)cc2)nc2c1CSCC2,single protein format,Kd,0,Oc1nc(-c2ccc(C(F)(F)F)cc2)nc2c1CSCC2
1,CHEMBL81977,single protein format,IC50,0,NC(=O)c1cccc(N)c1,single protein format,Ki,0,NC(=O)c1cccc(N)c1
2,CHEMBL121765,single protein format,IC50,0,Cc1nc2c(C(N)=O)cccc2[nH]1,single protein format,Ki,0,Cc1nc2c(C(N)=O)cccc2[nH]1
3,CHEMBL372303,single protein format,IC50,0,CN(C)CC(=O)Nc1ccc2[nH]c(=O)c3ccccc3c2c1,single protein format,Kd,0,CN(C)CC(=O)Nc1ccc2[nH]c(=O)c3ccccc3c2c1


In [134]:
dup= final_ic50.merge(final_cell, on = 'IDs', how = 'inner', validate = 'many_to_one')
dup

,IDs,New_BAO_Label_x,New_Standard_Type_x,binary_activity_x,Canonical_Smiles_x,New_BAO_Label_y,New_Standard_Type_y,binary_activity_y,Canonical_Smiles_y
0,CHEMBL571285,single protein format,IC50,1,NC(=O)c1cccc2[nH]c(-c3ccc(-c4cccnc4)cc3)nc12,cell-based format,EC50,1,NC(=O)c1cccc2[nH]c(-c3ccc(-c4cccnc4)cc3)nc12
1,"CHEMBL190434, CHEMBL426270",single protein format,IC50,1,O=S(=O)(NCCCN1CCOCC1)c1ccc2c(c1)Cc1c-2nc(O)c2c...,cell-based format,EC50,1,O=S(=O)(NCCCN1CCOCC1)c1ccc2c(c1)Cc1c-2nc(O)c2c...
2,CHEMBL3219283,single protein format,IC50,1,CN(C)Cc1cc2c([nH]c(=O)c3cccc(O)c32)s1,cell-based format,EC50,1,CN(C)Cc1cc2c([nH]c(=O)c3cccc(O)c32)s1
3,CHEMBL372303,single protein format,IC50,0,CN(C)CC(=O)Nc1ccc2[nH]c(=O)c3ccccc3c2c1,cell-based format,EC50,1,CN(C)CC(=O)Nc1ccc2[nH]c(=O)c3ccccc3c2c1
4,CHEMBL3735979,single protein format,IC50,1,O=C(c1ccc(I)cc1)N1CCN(C(=O)c2cc(Cc3n[nH]c(=O)c...,cell-based format,EC50,1,O=C(c1ccc(I)cc1)N1CCN(C(=O)c2cc(Cc3n[nH]c(=O)c...
5,CHEMBL576423,single protein format,IC50,0,NC(=O)c1cccc2[nH]c(-c3ccc(-c4ccccn4)cc3)nc12,cell-based format,EC50,1,NC(=O)c1cccc2[nH]c(-c3ccc(-c4ccccn4)cc3)nc12
6,CHEMBL521686,single protein format,IC50,1,O=C(c1cc(Cc2n[nH]c(=O)c3ccccc23)ccc1F)N1CCN(C(...,cell-based format,EC50,1,O=C(c1cc(Cc2n[nH]c(=O)c3ccccc23)ccc1F)N1CCN(C(...


In [135]:
dup= final_kikd.merge(final_cell, on = 'IDs', how = 'inner', validate = 'many_to_one')
dup

,IDs,New_BAO_Label_x,New_Standard_Type_x,binary_activity_x,Canonical_Smiles_x,New_BAO_Label_y,New_Standard_Type_y,binary_activity_y,Canonical_Smiles_y
0,CHEMBL541744,single protein format,Ki,0,CN(C)Cc1ccc2c(c1)NC(=O)C1CCCCN21,cell-based format,EC50,1,CN(C)Cc1ccc2c(c1)NC(=O)C1CCCCN21
1,CHEMBL372303,single protein format,Kd,0,CN(C)CC(=O)Nc1ccc2[nH]c(=O)c3ccccc3c2c1,cell-based format,EC50,1,CN(C)CC(=O)Nc1ccc2[nH]c(=O)c3ccccc3c2c1


Молекулы CHEMBL541744, CHEMBL372303 и CHEMBL576423 подозрительны

In [139]:
f = pd.concat([final_ic50, final_cell, final_kikd])

In [140]:
cell_dup = []

for k in set(final_ic50['IDs'].to_list()):
    if k in set(final_kikd['IDs'].to_list()):
        cell_dup.append(k)

In [141]:
f.head()

,IDs,New_BAO_Label,New_Standard_Type,binary_activity,Canonical_Smiles
0,CHEMBL124815,single protein format,IC50,0,O=C1NC(CCCl)Oc2ccccc21
1,CHEMBL557175,single protein format,IC50,1,NCc1nc(-c2ccc3[nH]c4c5c(c6c(c4c3c2)C(=O)NC6=O)...
2,CHEMBL250851,single protein format,IC50,1,Cc1cccc2c(=O)[nH]c(C3=CC(N4CCN(c5ccc(Cl)cc5)CC...
3,CHEMBL248648,single protein format,IC50,1,O=c1[nH]c(C2=C[C@@H](N3CCC(c4ccccc4)CC3)CC2)nc...
4,CHEMBL250849,single protein format,IC50,1,Cc1cccc2c(=O)[nH]c(C3=CC(N4CCC(c5ccc(F)cc5)CC4...


In [142]:
print(len(f))

f.sort_values(by = ['IDs'], inplace = True)

2425


In [143]:
#Сгруппируем по айдишникам и опишем результаты
diff = f.groupby('IDs')['binary_activity'].describe().reset_index().sort_values('count', ascending = False)

In [144]:
#Дропнем то, у чего одному айдишнику соответствует и активный, и неактивный р-т
to_drop = diff[~(diff['max'] == diff['min'])]['IDs'].to_list()
print(to_drop)

f  = f[~f['IDs'].isin(to_drop)]
print(len(f))

['CHEMBL372303', 'CHEMBL541744', 'CHEMBL576423']
2418


In [145]:
f.drop_duplicates(keep = 'first', inplace = True)
print(len(f))

f.drop_duplicates(subset = 'IDs', keep = 'first', inplace = True)
print(len(f))

2418
2410


In [147]:
f.Canonical_Smiles.nunique()

2410

In [146]:
f.to_csv('~/Chembl/Chembl_binary_activity_labels.csv')